In [1]:
import pandas as pd
import numpy as np

In [5]:
df_edn_level = pd.read_csv('./raw-data/Kiron Supplementary Data.xlsx - Educational Level of Migrants.csv')
df_over_qual = pd.read_csv('./raw-data/Kiron Supplementary Data.xlsx - Over qualification.csv')
df_popn = pd.read_csv('./raw-data/Kiron Supplementary Data.xlsx - Population in Education.csv')
df_students = pd.read_csv('./raw-data/kiron_students_cleaned.xlsx - kiron students cleaned.csv')

In [ ]:
class DataProcessing():
    
    def __init__(self):
       """
       """
    def clean(self, df):
        
        df = df.rename(columns=lambda x: x.strip()) # remove trailing whitespaces from column names
        
        # clean up 'attended_university' categorization
        df['attended_university'] = df['attended_university'].replace('TRUE\t', 'TRUE')
        df['attended_university'] = df['attended_university'].replace('FALSE\t', 'FALSE')
        df['attended_university'] = df['attended_university'].replace('\t', 'UNKNOWN')
        df['attended_university'] = df['attended_university'].fillna('UNKNOWN')
        
        # Replace country names to match geo.json file 
        df['nationality'] = df['nationality'].replace('Syrian Arab Republic', 'Syria')
        df['nationality'] = df['nationality'].replace('Iran (Islamic Republic of)', 'Iran')
        df['nationality'] = df['nationality'].replace('Palestine, State of', 'Palestine')
        df['nationality'] = df['nationality'].replace('Congo (Democratic Republic of the)', 'Dem. Rep. Congo')
        df['nationality'] = df['nationality'].replace('Russian Federation', 'Russia')
        df['nationality'] = df['nationality'].replace('Venezuela (Bolivarian Republic of)', 'Venezuela')
        df['nationality'] = df['nationality'].replace('Tanzania\tUnited Republic of', 'Tanzania')
        df['nationality'] = df['nationality'].replace('United Kingdom of Great Britain and Northern Ireland', 'United Kingdom')
        df['nationality'] = df['nationality'].replace("C√¥te d'Ivoire", "Côte d'Ivoire")
        df['nationality'] = df['nationality'].replace('United States of America', 'United States')
        df['nationality'] = df['nationality'].replace('South Sudan', 'S. Sudan')
        df['nationality'] = df['nationality'].replace('Western Sahara', 'W. Sahara')
        
        # convert string dates to datetime format
        df['student_since'] = df['student_since'].apply(lambda x : datetime.strptime(x,  "%m/%d/%Y"))
        df = df.sort_values('student_since', ascending=True)
        
        return df.reset_index()

    def aggregate_count(self, df):

        g = df.groupby('nationality')['index'].count()
        country_count = pd.DataFrame(g).reset_index().rename(columns={'index': 'count'})
        country_pcts = pd.DataFrame(g).apply(lambda x: 100 * x / float(x.sum())).reset_index().rename(columns={'index': 'pct'})
        country_stats = pd.merge(country_count, country_pcts, on='nationality', how='left')
        country_stats = country_stats.rename(columns={'nationality': 'country'})
        
        return country_stats.sort_values('pct', ascending=False)
    
    

In [ ]:
df_students = pd.read_csv('./raw-data/kiron_students_cleaned.xlsx - kiron students cleaned.csv')
dp = DataProcessing()
df_students_clean = dp.clean(df_students)
country_stats = dp.aggregate_count(df_students_clean)

In [ ]:
df_students_clean.to_csv('./processed-data/students_processed.csv', index=False)
country_stats.to_csv('./processed-data/country_stats.csv', index=False)

In [ ]:
with open('./raw-data/map.geo.json') as json_file:  
    map_data = json.load(json_file)

In [ ]:
for i in map_data['features']:
    print(i['properties']['name'])